In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time 
from unidecode import unidecode
import json 
from thefuzz import process,fuzz
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.ui import WebDriverWait

import requests

def getProxy():
    keydepip = '26191c8fd4436643e7e3488c5c0bb0e8'
    authen_ip = "59.153.238.246"
    url = f'http://api.fasthideproxy.com/socks5?apikey={keydepip}&ip={authen_ip}&changenow=1'
    print(url)
    response = requests.get(url, timeout=50).json()
    proxies = {}
    print(response)
    ip = ""
    if response['code'] == 1:
        ip = response['message']

    elif response['code'] == 0:
        doi = response['message'].split(' ')[1]
        
    return (ip)


def initDriver():
    proxy_data = getProxy()
    prox = Proxy()
    prox.proxy_type = ProxyType.MANUAL
    prox.socks_version = 5
    prox.socks_proxy = proxy_data
    WINDOW_SIZE = "1000,2000"
    chrome_options = Options()
    chrome_options.proxy = prox
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-infobars')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver



def find_element(driver, selectors):
    def expand_shadow_element(driver, element):
        return driver.execute_script('return arguments[0].shadowRoot', element)
    element = driver.find_element(By.CSS_SELECTOR,selectors[0])
    for selector in selectors[1:]:
        element = expand_shadow_element(driver, element) or element
        element = element.find_element(By.CSS_SELECTOR,selector)
    return element


def wait_for_elem(driver, selectors, timeout = 15):
    try:
        return WebDriverWait(driver, timeout).until(lambda driver: find_element(driver, selectors))
    except:
        raise RuntimeError(f"Element not found: {selectors}")



def check_similar(q,obj1):
    temp = (process.extract(q, obj1["features"], scorer=fuzz.ratio))
    print(temp)
    return len(temp)

driver = initDriver()

from selenium.webdriver.common.keys import Keys


domain = 'https://www.tripadvisor.com.vn'

question  = "khách sạn giá dưới 3 triệu, có bể bơi trên tầng thượng, phòng hút thuốc lá, phục vụ bữa sáng, có quầy bar. Tôi thích uống rượu vang và đi tắm hơi nên muốn có nhà tắm hơi"


data  = open("data.txt", "r").read().split("\n")
result = []
count = 0
WAIT_TIME = 7

for item in data:
    if(count < 15):
        try:
            target = domain + item
            # target = "https://www.tripadvisor.com.vn/Hotel_Review-g293925-d15113167-Reviews-Au_Lac_Charner_Hotel-Ho_Chi_Minh_City.html"
            print("[log] ", target)
            driver.get(target)
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            # listall = soup.findAll('div', {'class','uqMDf z BGJxv YGfmd YQkjl'})
            listall = soup.find('div', {'class','ui_columns MXlSZ'}).findAll('div', {'class','ui_column  '})
              
            try:
                rateing_overall = wait_for_elem(driver, ['main', '.kJyXc.P']).text
            except :
                rateing_overall = soup.find('span', {'class','uwJeR P'}).text
                
            # price = wait_for_elem(driver, ['main', '.VRono.Gi.B1.Z.BB.Pk.PY.Px.PK', '.NalGx','.aLfMd']).text
            price = wait_for_elem(driver, ['#MAIN', '.XQgaU', '.biGQs._P.fiohW.uuBRH','.gbXAQ']).text
            WebDriverWait(driver, WAIT_TIME).until(lambda x: price != '')
            if(listall):
                # tiennghi = listall[1].findAll('div', {'class','gFttI f ME Ci H3 _c'})
                tiennghi = listall[1].findAll('div', {'class','yplav f ME H3 _c'})
                feature = []
                for item in tiennghi:
                    feature.append(item.text)
            else:
                tiennghi = []
            obj = {
                "name_hotel": soup.find('h1', {'class','biGQs _P rRtyp'}).text,
                # "address" : soup.find('span', {'class','CdhWK _S'}).text, 
                "address" : soup.find('span', {'class','biGQs _P pZUbB KxBGd'}).text, 
                "rating" : rateing_overall,
                "price" : price,
                "features" : feature
            }
            print(obj)
            sim = check_similar(question,obj)
            print(obj['name_hotel'], sim)
            result.append(obj)
            count += 1
        except Exception as e:
            print(e)
            break
            
    else:
        count = 0
        driver.close()
        initDriver()






http://api.fasthideproxy.com/socks5?apikey=26191c8fd4436643e7e3488c5c0bb0e8&ip=59.153.238.246&changenow=1
{'code': 0, 'message': 'Còn 2 giây mới được request'}
[log]  https://www.tripadvisor.com.vn/Hotel_Review-g293925-d25442757-Reviews-La_Siesta_Premium_Sai_Gon-Ho_Chi_Minh_City.html
'NoneType' object has no attribute 'findAll'


In [32]:
soup.findAll('div', {'class','ui_columns MXlSZ'})[1].text

IndexError: list index out of range

In [23]:
wait_for_elem(driver, ['#MAIN', '.XQgaU', '.biGQs._P.fiohW.uuBRH','.gbXAQ']).text

'4.262.400 ₫'

In [6]:
soup.find('span', {'class','uwJeR P'}).text

'5,0'

In [ ]:
# pip install thefuzz

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests_html import HTMLSession

session = HTMLSession()

url = "https://www.tripadvisor.com/Hotel_Review-g293925-d306225-Reviews-LOTTE_HOTEL_SAIGON-Ho_Chi_Minh_City.html"


def fetchAndSaveData(url, path):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36',
            "method": "GET",
            "mode": "cors"
             
             }
    r = requests.Session()
    r = r.get(url)
 
    
    # r = requests.get(url,headers=headers)
    
    print(r)
    
    with open(path, "w", encoding='utf-8') as f:
        f.write(r.text)

fetchAndSaveData(url, "data/trip.html")


with open('data/trip.html', 'r', encoding='utf-8') as f:
    data = f.read()

soup = BeautifulSoup(data, "lxml")

# print(soup.prettify())

hotel_name = soup.find('h1', class_="biGQs").text
print(hotel_name)

hotel_rating = soup.find('span', class_="uwJeR").text
print(hotel_rating)

hotel_address = soup.find('span', class_="oAPmj").text
print("Hotel Address", hotel_address)

total_reviews = soup.find('span', class_="hkxYU").text
# print(total_reviews)

TotalNoReviews = total_reviews.split(" ")[0]
TotalNoReviews = int(TotalNoReviews.replace(",", ""))
print(TotalNoReviews)

reviews = soup.find_all('div', class_="YibKl")

review_title = []
review_description = []

titles = soup.find_all('a', class_="Qwuub")

for title in titles:
    review_title.append(title.text)

# print(review_title)
print(len(review_title))



descriptions = soup.find_all('span', class_="QewHA")


for description in descriptions:
    review_description.append(description.text)

# print(review_description)
print(len(review_description))

reviews_author = []
reviews_title = []
reviews_description = []
reviews_date = []
reviews_type = []
reviews_response = []

container = soup.find_all("div", class_="YibKl")

for item in container:
    author = item.find('a', class_="ui_header_link").text
    title = item.find("div", class_="KgQgP").text
    description = item.find("span", class_="QewHA").text
    date = item.find("span", class_="teHYY").text.split(":")[1].strip()
    type=""
    if(item.find("span", class_="TDKzw")):
        type = item.find("span", class_="TDKzw").text.split(":")[1].strip()
            
    responseBox = item.find("div", class_="ajLyr")
    response = ""
    res = responseBox.find("span", class_="MInAm").text
        
    if(len(res) > 0):
        response = res
    
    ratingBox = item.find("span", class_="ui_bubble_rating")

    span_classes = ratingBox['class']
    
    if(len(span_classes) == 2):
        rating = span_classes[1].split("_")
        print(int(rating[1])/10) 

    
    reviews_author.append(author)
    reviews_title.append(title)
    reviews_description.append(description)
    reviews_date.append(date)
    reviews_type.append(type)
    reviews_response.append(response)
    
# Create a dictionary from the data
data = {
    "Hotel Name": [hotel_name] + [""] * (len(reviews_author) - 1),
    "Hotel Address": [hotel_address] + [""] * (len(reviews_author) - 1),
    "Hotel Rating": [hotel_rating] + [""] * (len(reviews_author) - 1),
    "Author": reviews_author,
    "Review Title": reviews_title,
    "Review Description": reviews_description,
    "type": reviews_type,
    "Date": reviews_date,
    "Response": reviews_response
}

# Create the DataFrame
df = pd.DataFrame(data)


# Print the DataFrame
# print(df)

df.to_csv("test.csv", index=False)

    

# i = 0
# for review in reviews:
#     i+=1
#     if(i == 4):
#         break
    
#     print(i)
#     title = reviews.find('a', class_="Qwuub")
#     print(title.text)